In [ ]:
# Load dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.1e}'.format
mport sys
pd.options.display.float_format = '{:,.1e}'.format
sys.path.insert(0,'../../../statistics_helper/')
from excel_utils import *

# Estimating the biomass of livestock
To estimate the biomass of livestock, we rely on data on global stocks of cattle, sheep goats, and pigs froms the Food and Agriculture Organization database FAOStat. We downloaded data from the domain Production/Live animals.
We combined data on the total stocks of each animal with estimates of the mean mass of each type of animal species (in kg) from [Dong et al.](http://www.ipcc-nggip.iges.or.jp/public/2006gl/pdf/4_Volume4/V4_10_Ch10_Livestock.pdf), Annex 10A.2, Tables 10A-4 to 10A-9.

Here are samples of the data:

In [1]:
# Load global stocks data
stocks = pd.read_csv('FAOSTAT_stock_data_mammals.csv')
stocks.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QA,Live Animals,5100,Africa,5111,Stocks,1107,Asses,2014,2014,Head,18946358,A,"Aggregate, may include official, semi-official..."
1,QA,Live Animals,5100,Africa,5111,Stocks,946,Buffaloes,2014,2014,Head,3949287,A,"Aggregate, may include official, semi-official..."
2,QA,Live Animals,5100,Africa,5111,Stocks,1126,Camels,2014,2014,Head,23533724,A,"Aggregate, may include official, semi-official..."
3,QA,Live Animals,5100,Africa,5111,Stocks,866,Cattle,2014,2014,Head,312327289,A,"Aggregate, may include official, semi-official..."
4,QA,Live Animals,5100,Africa,5111,Stocks,1016,Goats,2014,2014,Head,374380445,A,"Aggregate, may include official, semi-official..."


In [2]:
# Load species body mass data
body_mass = pd.read_excel('livestock_body_mass.xlsx',skiprows=1,index_col=0) 
body_mass.head()

,Cattle - dairy,Cattle - non-dairy,Buffaloes,Swine - market,Swine - breeding,Sheep,Goats,Horses,Asses,Mules,Camels,"Camelids, other"
IPCC Area,,,,,,,,,,,,
Indian Subcontinent,275,110,295,28,28,2.8e+01,3.0e+01,238,130,130,217,217
Eastern Europe,550,391,380,50,180,4.8e+01,3.8e+01,377,130,130,217,217
Africa,275,173,380,28,28,2.8e+01,3.0e+01,238,130,130,217,217
Oceania,500,330,380,45,180,4.8e+01,3.8e+01,377,130,130,217,217
Western Europe,600,420,380,50,198,4.8e+01,3.8e+01,377,130,130,217,217


We pivot the stocks DataFrame to have a view of each kind of animal at each region:

In [3]:
# Replace NaN with zeros
stocks.fillna(value=0,inplace=True)
stock_pivot = pd.pivot(stocks.Area,stocks.Item, stocks.Value).astype(float)

# Replace NaN with zeros
stock_pivot.fillna(value=0,inplace=True)

stock_pivot

Item,Asses,Buffaloes,"Camelids, other",Camels,Cattle,Goats,Horses,Mules,Pigs,Sheep
Area,,,,,,,,,,
Africa,1.9e+07,3.9e+06,0.0e+00,2.4e+07,3.1e+08,3.7e+08,6.1e+06,1.0e+06,3.4e+07,3.4e+08
Americas,6.8e+06,1.3e+06,8.9e+06,0.0e+00,5.1e+08,3.6e+07,3.3e+07,5.9e+06,1.7e+08,8.6e+07
Asia,1.6e+07,1.9e+08,0.0e+00,4.2e+06,4.9e+08,5.8e+08,1.4e+07,3.0e+06,5.9e+08,5.4e+08
Eastern Europe,1.0e+05,1.7e+04,0.0e+00,7.4e+03,4.0e+07,4.6e+06,2.8e+06,3.5e+03,5.3e+07,3.6e+07
Northern America,5.2e+04,0.0e+00,0.0e+00,0.0e+00,1.0e+08,2.6e+06,1.1e+07,4.0e+03,8.1e+07,6.1e+06
Oceania,9.0e+03,2.4e+02,0.0e+00,0.0e+00,4.0e+07,4.0e+06,4.0e+05,0.0e+00,5.3e+06,1.0e+08
Southern Asia,8.3e+06,1.5e+08,0.0e+00,1.7e+06,2.7e+08,2.9e+08,1.3e+06,5.8e+05,1.1e+07,1.5e+08
Western Europe,3.4e+04,6.5e+03,0.0e+00,0.0e+00,4.2e+07,2.1e+06,1.1e+06,3.1e+04,6.5e+07,1.1e+07


There is a difference between the body mass of a dairy producing cow to a non-dairy producing cow. We thus count seperately the dairy producing cattle from the non-dairy producing cattle. Data about the amount of dairy cattle comes from the FAOStat domain Production - Livestock Primary.
There is also a difference in body mass between breeding and non-breeding pigs. We assume 90% of the population is breeding based on IPCC, 2006, Vol.4, Ch.10,Table.10.19.

In [4]:
# Load data on the number of dairy producing cattle
dairy = pd.read_csv('FAOSTAT_cattle_dairy_data.csv')

# Set the index of the DataFrame to be the region so we can compare with the stocks data
dairy.set_index('Area',inplace=True)

# Add a category of dairy producing cattle
stock_pivot['Cattle - dairy'] = dairy.Value

# Set the amount of non-dairy producing cattle to be the total number minus the dairy producing cattle
stock_pivot['Cattle'] = stock_pivot['Cattle']-stock_pivot['Cattle - dairy']

# Rename the Cattle column name to Cattle - non-dairy
stock_pivot.rename(columns={'Cattle': 'Cattle - non-dairy'}, inplace=True)

# Set the amount of non-breeding (market) pigs (swine) to 10% of the total amount of pigs
stock_pivot['Swine - market'] = 0.1*stock_pivot['Pigs']

# Set the amount of breeding pigs (swine) to 90% of the total amount of pigs
stock_pivot['Pigs'] *= 0.9

# Rename the Pigs column name to Swine - breeding
stock_pivot.rename(columns={'Pigs': 'Swine - breeding'}, inplace=True)

stock_pivot

Item,Asses,Buffaloes,"Camelids, other",Camels,Cattle - non-dairy,Goats,Horses,Mules,Swine - breeding,Sheep,Cattle - dairy,Swine - market
Area,,,,,,,,,,,,
Africa,1.9e+07,3.9e+06,0.0e+00,2.4e+07,2.4e+08,3.7e+08,6.1e+06,1.0e+06,3.1e+07,3.4e+08,67436568,3.4e+06
Americas,6.8e+06,1.3e+06,8.9e+06,0.0e+00,4.5e+08,3.6e+07,3.3e+07,5.9e+06,1.5e+08,8.6e+07,54930519,1.7e+07
Asia,1.6e+07,1.9e+08,0.0e+00,4.2e+06,3.8e+08,5.8e+08,1.4e+07,3.0e+06,5.3e+08,5.4e+08,107571193,5.9e+07
Eastern Europe,1.0e+05,1.7e+04,0.0e+00,7.4e+03,2.3e+07,4.6e+06,2.8e+06,3.5e+03,4.8e+07,3.6e+07,16188776,5.3e+06
Northern America,5.2e+04,0.0e+00,0.0e+00,0.0e+00,9.1e+07,2.6e+06,1.1e+07,4.0e+03,7.3e+07,6.1e+06,10161310,8.1e+06
Oceania,9.0e+03,2.4e+02,0.0e+00,0.0e+00,3.3e+07,4.0e+06,4.0e+05,0.0e+00,4.8e+06,1.0e+08,6874751,5.3e+05
Southern Asia,8.3e+06,1.5e+08,0.0e+00,1.7e+06,2.0e+08,2.9e+08,1.3e+06,5.8e+05,1.0e+07,1.5e+08,69325063,1.1e+06
Western Europe,3.4e+04,6.5e+03,0.0e+00,0.0e+00,3.1e+07,2.1e+06,1.1e+06,3.1e+04,5.8e+07,1.1e+07,11289409,6.5e+06


Data on the mass of animals is divided into different regions than the FAOStat data so we need preprocess the stocks DataFrame and merge it with the body mass data:

In [5]:
# Preprocessing the stocks DataFrame

# Calculate the total number of animals in Latin America by subtracting values for Northern America from the total
# values for the Americas
stock_pivot.loc['Americas'] -= stock_pivot.loc['Northern America']

# Change name of Americas to Latin America
stock_pivot.rename(index={'Americas': 'Latin America'},inplace=True)

# Calculate the total number of animals in Asia without the Indian Subcontinent by subtracting values for the Southern Asia 
# from the total values for the Asia
stock_pivot.loc['Asia'] -= stock_pivot.loc['Southern Asia']

# Change name of Southern Asia to Indian Subcontinent
stock_pivot.rename(index={'Southern Asia': 'Indian Subcontinent'},inplace=True)


stock_pivot


Item,Asses,Buffaloes,"Camelids, other",Camels,Cattle - non-dairy,Goats,Horses,Mules,Swine - breeding,Sheep,Cattle - dairy,Swine - market
Area,,,,,,,,,,,,
Africa,1.9e+07,3.9e+06,0.0e+00,2.4e+07,2.4e+08,3.7e+08,6.1e+06,1.0e+06,3.1e+07,3.4e+08,6.7e+07,3.4e+06
Latin America,6.7e+06,1.3e+06,8.9e+06,0.0e+00,3.6e+08,3.3e+07,2.2e+07,5.9e+06,8.0e+07,8.0e+07,4.5e+07,8.9e+06
Asia,8.2e+06,3.7e+07,0.0e+00,2.5e+06,1.8e+08,2.9e+08,1.3e+07,2.4e+06,5.2e+08,3.8e+08,3.8e+07,5.8e+07
Eastern Europe,1.0e+05,1.7e+04,0.0e+00,7.4e+03,2.3e+07,4.6e+06,2.8e+06,3.5e+03,4.8e+07,3.6e+07,1.6e+07,5.3e+06
Northern America,5.2e+04,0.0e+00,0.0e+00,0.0e+00,9.1e+07,2.6e+06,1.1e+07,4.0e+03,7.3e+07,6.1e+06,1.0e+07,8.1e+06
Oceania,9.0e+03,2.4e+02,0.0e+00,0.0e+00,3.3e+07,4.0e+06,4.0e+05,0.0e+00,4.8e+06,1.0e+08,6.9e+06,5.3e+05
Indian Subcontinent,8.3e+06,1.5e+08,0.0e+00,1.7e+06,2.0e+08,2.9e+08,1.3e+06,5.8e+05,1.0e+07,1.5e+08,6.9e+07,1.1e+06
Western Europe,3.4e+04,6.5e+03,0.0e+00,0.0e+00,3.1e+07,2.1e+06,1.1e+06,3.1e+04,5.8e+07,1.1e+07,1.1e+07,6.5e+06


We now multiply the stocks of each animal type and for each region by the characteristic body weight of each animal:

In [6]:
wet_biomass =(body_mass*stock_pivot)
wet_biomass

,Asses,Buffaloes,"Camelids, other",Camels,Cattle - dairy,Cattle - non-dairy,Goats,Horses,Mules,Sheep,Swine - breeding,Swine - market
Africa,2.5e+09,1.5e+09,0.0e+00,5.1e+09,1.9e+10,4.2e+10,1.1e+10,1.4e+09,1.3e+08,9.5e+09,8.7e+08,9.6e+07
Asia,1.1e+09,1.4e+10,0.0e+00,5.5e+08,1.3e+10,7.2e+10,1.1e+10,4.9e+09,3.1e+08,1.9e+10,9.4e+10,2.9e+09
Eastern Europe,1.3e+07,6.3e+06,0.0e+00,1.6e+06,8.9e+09,9.2e+09,1.8e+08,1.0e+09,4.6e+05,1.8e+09,8.6e+09,2.6e+08
Indian Subcontinent,1.1e+09,4.5e+10,0.0e+00,3.7e+08,1.9e+10,2.2e+10,8.8e+09,3.2e+08,7.6e+07,4.3e+09,2.9e+08,3.2e+07
Latin America,8.8e+08,5.0e+08,1.9e+09,0.0e+00,1.8e+10,1.1e+11,9.9e+08,5.2e+09,7.7e+08,2.2e+09,2.2e+09,2.5e+08
Middle east,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Northern America,6.8e+06,0.0e+00,0.0e+00,0.0e+00,6.1e+09,3.5e+10,1.0e+08,4.0e+09,5.2e+05,3.0e+08,1.4e+10,3.7e+08
Oceania,1.2e+06,9.3e+04,0.0e+00,0.0e+00,3.4e+09,1.1e+10,1.5e+08,1.5e+08,0.0e+00,5.0e+09,8.7e+08,2.4e+07
Western Europe,4.4e+06,2.5e+06,0.0e+00,0.0e+00,6.8e+09,1.3e+10,7.9e+07,4.1e+08,4.1e+06,5.2e+08,1.2e+10,3.2e+08


We sum over all regions and convert units from kg wet weight to Gt C carbon by assuming carbon is ≈15% of the wet weight (30% dry weight of wet weight and carbon is 50% of dry weight).

In [7]:
pd.options.display.float_format = '{:,.3f}'.format

# conversion factor from kg wet weight to Gt C
kg_to_gt_c = 1000*0.15/1e15
total_biomass = wet_biomass.sum()*kg_to_gt_c
total_biomass

Asses                0.001
Buffaloes            0.009
Camelids, other      0.000
Camels               0.001
Cattle - dairy       0.014
Cattle - non-dairy   0.047
Goats                0.005
Horses               0.003
Mules                0.000
Sheep                0.006
Swine - breeding     0.020
Swine - market       0.001
dtype: float64

We sum over all animal categories to generate our best estimate for the total biomass of livestock

In [1]:
best_estimate = total_biomass.sum()
print('Our best estimate for the biomass of mammal livestock is %.1f Gt C' % best_estimate)

NameError: name 'total_biomass' is not defined

In [ ]:
# Feed results to the chordate biomass data
old_results = pd.read_excel('../../animal_biomass_estimate.xlsx',index_col=0)
result = old_results.copy()
result.loc['Livestock',(['Biomass [Gt C]','Uncertainty'])] = (best_estimate,None)
result.to_excel('../../animal_biomass_estimate.xlsx')

# Feed results to Table 1 & Fig. 1
update_results(sheet='Table1 & Fig1', 
               row=('Animals','Livestock'), 
               col='Biomass [Gt C]',
               values=best_estimate,
               path='../../../results.xlsx')

# Feed results to Table S1
update_results(sheet='Table S1', 
               row=('Animals','Livestock'), 
               col='Number of individuals',
               values=stock_pivot.sum().sum(),
               path='../../../results.xlsx')